In [ ]:
%%time

import os.path
import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar
from tqdm import tqdm
from tqdm.dask import TqdmCallback

from shared_code.utility.spark.set_environ import set_azure_env

cb = TqdmCallback(desc="global")
cb.register()

tqdm.pandas()
tqdm.pandas(desc="global")

pbar = ProgressBar()
pbar.register()

set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter

fs_adapter = AzureFileStorageAdapter('data')
file_system = fs_adapter.get_file_storage()

In [ ]:
from typing import List
from dataclasses import dataclass


@dataclass
class DataSources:
	name: str
	data: List[str]

	@staticmethod
	def from_dict(obj: dict) -> 'DataSources':
		_name = obj.get("name")
		_data = [x for x in obj.get("data")]
		return DataSources(_name, _data)

In [ ]:
def add_source(x: object, source_list) -> str:
	for source in source_list:
		if x['subreddit'] in source['data']:
			return source['name']
	return ""

In [ ]:
def add_compressed_data(x: object) -> str:
	try:
		data = open(x['path'], 'rb').read()
		return data
	except Exception as e:
		print(e)
		return ""

In [ ]:
%%time
all_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(all_data)

In [ ]:
%%time

filtered_on_exist = all_data.where(lambda x: x['exists']).dropna(how='all').reset_index(drop=True)
filtered_on_exist = filtered_on_exist.where(lambda x: x['caption'] != "").dropna(how='all').reset_index(drop=True)
display(filtered_on_exist)

In [ ]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes","trippinthroughtime"]},
	{"name": "SexyDiffusion", "data": ["sfwpetite","selfies","Amicute","amihot","AmIhotAF","HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]}
]
sources_df = pd.DataFrame.from_records(sources)
display(sources_df)

In [ ]:
%%time

smaller_exportable_df = pd.DataFrame(
	data=[filtered_on_exist['path'], filtered_on_exist['image_name'], filtered_on_exist['caption'], filtered_on_exist['title'], filtered_on_exist['subreddit']],
	index=['path', 'image_name', 'caption', 'title', 'subreddit']).T

with ProgressBar():
	smaller_exportable_df['name'] = smaller_exportable_df.progress_apply(lambda x: add_source(x, sources), axis=1)

display(smaller_exportable_df)

In [ ]:
%%time

with cb:
	dask_frame = dd.from_pandas(smaller_exportable_df, npartitions=10)
	smaller_exportable_df['image_data'] = dask_frame.apply(lambda x: add_compressed_data(x), meta=('str', object), axis=1).compute()

display(smaller_exportable_df)

In [ ]:
%%time

filtered_again = smaller_exportable_df.where(lambda x: x['image_data'] != "").dropna(how='all').reset_index(drop=True)
filtered_again = filtered_again.where(lambda x: x['name'] != "").dropna(how='all').reset_index(drop=True)
display(filtered_again)

In [ ]:
%%time

final = filtered_again
display(final)

grouped = final.groupby('name')
groupings = [grouped.get_group(x) for x in grouped.groups]

for group in groupings:
	group.to_parquet(f"data/curated/{group['name'].iloc[0]}.parquet")

In [ ]:
lines = []
training_lines = []
for record in final.to_dict(orient='records'):
	subreddit = record['subreddit']
	name = record['name']
	prompt = record['title']
	caption = record['caption']
	line = f"<|startoftext|><|model|>{name}<|prompt|>{prompt}<|text|>{caption}<|endoftext|>" + "\n"
	lines.append(line)
with open("training.txt", "wb") as f:
	for line in lines:
		f.write(line.encode("utf-8"))

In [ ]:
data_paths = os.listdir("data/curated")

dfs = []

for path in data_paths:
	foo = os.path.join("data", "curated", path)
	df = pd.read_parquet(foo)
	dfs.append(df)

final = pd.concat(dfs)
display(final)